# Lab 2: Local Search Algorithms

## Learning Objectives

By the end of this lab, you will:
- Understand local search principles
- Implement Hill Climbing and variants
- Master Simulated Annealing
- Apply local search to classic problems (TSP, N-Queens)
- Know when to use each algorithm
- Escape local optima effectively

## What is Local Search?

**Local Search** algorithms explore the search space by moving from one solution to nearby "neighbor" solutions.

**Key Idea**: 
- Start with a solution
- Look at neighbors
- Move to a better neighbor
- Repeat until no improvement

**Advantages**:
- Memory efficient (only store current solution)
- Work well for large state spaces
- Often find good solutions quickly

**Disadvantages**:
- Can get stuck in local optima
- No guarantee of optimality
- Need good neighborhood definition

## Real-World Applications

- 🚚 **Vehicle Routing**: Optimize delivery routes
- 📅 **Scheduling**: Course timetables, shift scheduling
- 🎨 **VLSI Design**: Circuit layout optimization
- 🧬 **Protein Folding**: Finding stable conformations
- 📱 **Network Design**: Optimize network topology


In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, List, Tuple, Optional
import time
from copy import deepcopy
import random

# Set random seed
np.random.seed(42)
random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: Hill Climbing

### Basic Hill Climbing Algorithm

**Algorithm**:
```
1. Start with a random solution
2. Generate all neighbors
3. If best neighbor is better than current:
   - Move to best neighbor
   - Go to step 2
4. Else: return current solution (local optimum)
```

**Like climbing a mountain in fog** - always go uphill, stop when nowhere to go up.


In [ ]:
class HillClimbing:
    """Basic Hill Climbing optimizer."""
    
    def __init__(self, objective_fn: Callable, neighbor_fn: Callable, 
                 maximize: bool = False):
        """
        Initialize Hill Climbing.
        
        Args:
            objective_fn: Function to optimize
            neighbor_fn: Function that generates neighbors of a solution
            maximize: If True, maximize instead of minimize
        """
        self.objective_fn = objective_fn
        self.neighbor_fn = neighbor_fn
        self.maximize = maximize
    
    def is_better(self, new_val: float, old_val: float) -> bool:
        """Check if new value is better than old."""
        if self.maximize:
            return new_val > old_val
        return new_val < old_val
    
    def optimize(self, initial_solution, max_iterations: int = 1000) -> Tuple:
        """
        Run hill climbing.
        
        Returns:
            (best_solution, best_value, history)
        """
        current = initial_solution
        current_value = self.objective_fn(current)
        history = [current_value]
        
        for iteration in range(max_iterations):
            # Generate neighbors
            neighbors = self.neighbor_fn(current)
            
            if not neighbors:
                break
            
            # Evaluate all neighbors
            neighbor_values = [(n, self.objective_fn(n)) for n in neighbors]
            
            # Find best neighbor
            best_neighbor, best_neighbor_value = min(
                neighbor_values, 
                key=lambda x: x[1] if not self.maximize else -x[1]
            )
            
            # If best neighbor is better, move to it
            if self.is_better(best_neighbor_value, current_value):
                current = best_neighbor
                current_value = best_neighbor_value
                history.append(current_value)
            else:
                # Local optimum reached
                break
        
        return current, current_value, history


# Example: Optimize 1D function
def simple_1d_function(x):
    """Simple 1D function with multiple local minima."""
    return x * np.sin(x) + 0.1 * x**2

def get_1d_neighbors(x, step=0.1):
    """Generate neighbors for 1D continuous optimization."""
    return [x - step, x + step]

print("Hill Climbing on 1D Function")
print("=" * 60)

# Try different starting points
starting_points = [0.5, 3.0, 8.0]

plt.figure(figsize=(14, 6))

# Plot function
x_range = np.linspace(0, 10, 1000)
y_range = [simple_1d_function(x) for x in x_range]
plt.plot(x_range, y_range, 'b-', linewidth=2, alpha=0.5, label='Objective function')

colors = ['red', 'green', 'orange']
for start_x, color in zip(starting_points, colors):
    hc = HillClimbing(simple_1d_function, get_1d_neighbors, maximize=False)
    best_x, best_val, history = hc.optimize(start_x, max_iterations=100)
    
    print(f"Starting at x={start_x:.1f}:")
    print(f"  Found minimum at x={best_x:.3f}, f(x)={best_val:.3f}")
    print(f"  Iterations: {len(history)}")
    print()
    
    # Plot start and end points
    plt.plot(start_x, simple_1d_function(start_x), 'o', 
            color=color, markersize=12, label=f'Start {start_x:.1f}')
    plt.plot(best_x, best_val, '*', color=color, markersize=20,
            markeredgecolor='black', markeredgewidth=2)

plt.xlabel('x', fontsize=12, fontweight='bold')
plt.ylabel('f(x)', fontsize=12, fontweight='bold')
plt.title('Hill Climbing with Different Starting Points', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Notice: Different starting points lead to different local minima!")

## Part 2: Hill Climbing Variants

### Variants to Improve Performance

1. **Steepest-Ascent**: Examine all neighbors (what we did above)
2. **First-Choice**: Take first improving neighbor
3. **Random-Restart**: Run multiple times with different starts
4. **Stochastic**: Sometimes accept worse moves


In [ ]:
class RandomRestartHillClimbing:
    """Hill Climbing with random restarts to escape local optima."""
    
    def __init__(self, objective_fn: Callable, neighbor_fn: Callable,
                 initial_solution_fn: Callable, maximize: bool = False):
        """
        Initialize Random Restart Hill Climbing.
        
        Args:
            objective_fn: Function to optimize
            neighbor_fn: Generate neighbors
            initial_solution_fn: Generate random initial solutions
            maximize: If True, maximize instead of minimize
        """
        self.hc = HillClimbing(objective_fn, neighbor_fn, maximize)
        self.initial_solution_fn = initial_solution_fn
    
    def optimize(self, n_restarts: int = 10, max_iterations: int = 1000) -> Tuple:
        """
        Run hill climbing with random restarts.
        
        Returns:
            (best_solution, best_value, all_histories)
        """
        best_solution = None
        best_value = np.inf if not self.hc.maximize else -np.inf
        all_histories = []
        
        for restart in range(n_restarts):
            # Generate random starting point
            initial = self.initial_solution_fn()
            
            # Run hill climbing
            solution, value, history = self.hc.optimize(initial, max_iterations)
            all_histories.append(history)
            
            # Update best
            if self.hc.is_better(value, best_value):
                best_value = value
                best_solution = solution
        
        return best_solution, best_value, all_histories


# Test random restart
def random_initial_1d():
    """Generate random starting point."""
    return np.random.uniform(0, 10)

print("Random Restart Hill Climbing")
print("=" * 60)

rrhc = RandomRestartHillClimbing(
    simple_1d_function, 
    get_1d_neighbors, 
    random_initial_1d,
    maximize=False
)

best_x, best_val, histories = rrhc.optimize(n_restarts=20, max_iterations=100)

print(f"After 20 restarts:")
print(f"Best solution: x={best_x:.4f}")
print(f"Best value: f(x)={best_val:.4f}")
print()

# Visualize all runs
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot objective function and best solution
x_range = np.linspace(0, 10, 1000)
y_range = [simple_1d_function(x) for x in x_range]
ax1.plot(x_range, y_range, 'b-', linewidth=2, alpha=0.7)
ax1.plot(best_x, best_val, 'r*', markersize=25, 
        markeredgecolor='darkred', markeredgewidth=2, label='Best found')
ax1.set_xlabel('x', fontweight='bold', fontsize=12)
ax1.set_ylabel('f(x)', fontweight='bold', fontsize=12)
ax1.set_title('Best Solution Found', fontweight='bold', fontsize=13)
ax1.legend()
ax1.grid(alpha=0.3)

# Plot convergence of all restarts
for i, history in enumerate(histories):
    ax2.plot(history, alpha=0.5, linewidth=1)
ax2.set_xlabel('Iteration', fontweight='bold', fontsize=12)
ax2.set_ylabel('Objective Value', fontweight='bold', fontsize=12)
ax2.set_title('Convergence of All Restarts', fontweight='bold', fontsize=13)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Random restarts help find better solutions!")

## Part 3: Simulated Annealing

### The Algorithm

**Key Insight**: Sometimes accept worse moves to escape local optima!

**Inspired by**: Metallurgy - slowly cooling metal to reach low-energy state

**Algorithm**:
```
1. Start with initial solution and high temperature T
2. Generate random neighbor
3. If neighbor is better: accept
4. If neighbor is worse: accept with probability e^(-ΔE/T)
5. Decrease temperature
6. Repeat until temperature is low
```

**Temperature schedule**: Controls exploration vs exploitation
- High T: Accept most moves (explore)
- Low T: Accept only improvements (exploit)


In [ ]:
class SimulatedAnnealing:
    """Simulated Annealing optimizer."""
    
    def __init__(self, objective_fn: Callable, neighbor_fn: Callable,
                 initial_temp: float = 100.0, cooling_rate: float = 0.95,
                 min_temp: float = 0.01, maximize: bool = False):
        """
        Initialize Simulated Annealing.
        
        Args:
            objective_fn: Function to optimize
            neighbor_fn: Generate random neighbor
            initial_temp: Starting temperature
            cooling_rate: Temperature decay factor (0 < α < 1)
            min_temp: Minimum temperature (stopping criterion)
            maximize: If True, maximize instead of minimize
        """
        self.objective_fn = objective_fn
        self.neighbor_fn = neighbor_fn
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp
        self.maximize = maximize
    
    def acceptance_probability(self, old_cost: float, new_cost: float, 
                              temperature: float) -> float:
        """Calculate probability of accepting worse solution."""
        if self.maximize:
            delta = new_cost - old_cost
        else:
            delta = old_cost - new_cost
        
        if delta > 0:
            return 1.0  # Always accept better solution
        else:
            return np.exp(delta / temperature)
    
    def optimize(self, initial_solution, max_iterations: int = 10000) -> Tuple:
        """
        Run simulated annealing.
        
        Returns:
            (best_solution, best_value, history, temp_history)
        """
        current = initial_solution
        current_cost = self.objective_fn(current)
        
        best_solution = current
        best_cost = current_cost
        
        temperature = self.initial_temp
        
        history = [current_cost]
        best_history = [best_cost]
        temp_history = [temperature]
        acceptance_history = []
        
        iteration = 0
        while temperature > self.min_temp and iteration < max_iterations:
            # Generate neighbor
            neighbor = self.neighbor_fn(current)
            neighbor_cost = self.objective_fn(neighbor)
            
            # Calculate acceptance probability
            accept_prob = self.acceptance_probability(current_cost, neighbor_cost, temperature)
            
            # Accept or reject
            if np.random.random() < accept_prob:
                current = neighbor
                current_cost = neighbor_cost
                acceptance_history.append(1)
                
                # Update best
                if (self.maximize and current_cost > best_cost) or \
                   (not self.maximize and current_cost < best_cost):
                    best_solution = current
                    best_cost = current_cost
            else:
                acceptance_history.append(0)
            
            # Cool down
            temperature *= self.cooling_rate
            
            # Record history
            history.append(current_cost)
            best_history.append(best_cost)
            temp_history.append(temperature)
            
            iteration += 1
        
        return best_solution, best_cost, {
            'current': history,
            'best': best_history,
            'temperature': temp_history,
            'acceptance': acceptance_history
        }


# Test simulated annealing on 1D function
def get_random_1d_neighbor(x, temperature=None):
    """Generate random neighbor (single value)."""
    step = np.random.normal(0, 0.5)  # Random step
    return x + step

print("Simulated Annealing on 1D Function")
print("=" * 60)

# Compare starting from poor location
start_x = 8.0

# Hill Climbing
hc = HillClimbing(simple_1d_function, get_1d_neighbors, maximize=False)
hc_sol, hc_val, hc_hist = hc.optimize(start_x)

# Simulated Annealing
sa = SimulatedAnnealing(
    simple_1d_function, 
    get_random_1d_neighbor,
    initial_temp=10.0,
    cooling_rate=0.99,
    maximize=False
)
sa_sol, sa_val, sa_hist = sa.optimize(start_x, max_iterations=1000)

print(f"Starting from x={start_x}:")
print(f"\nHill Climbing:")
print(f"  Final: x={hc_sol:.4f}, f(x)={hc_val:.4f}")
print(f"\nSimulated Annealing:")
print(f"  Final: x={sa_sol:.4f}, f(x)={sa_val:.4f}")
print()

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Objective function
x_range = np.linspace(0, 10, 1000)
y_range = [simple_1d_function(x) for x in x_range]

# Plot 1: Solutions
ax = axes[0, 0]
ax.plot(x_range, y_range, 'b-', linewidth=2, alpha=0.5)
ax.plot(start_x, simple_1d_function(start_x), 'go', markersize=12, label='Start')
ax.plot(hc_sol, hc_val, 'rs', markersize=15, label=f'HC: {hc_val:.3f}')
ax.plot(sa_sol, sa_val, 'r*', markersize=20, label=f'SA: {sa_val:.3f}')
ax.set_xlabel('x', fontweight='bold')
ax.set_ylabel('f(x)', fontweight='bold')
ax.set_title('Final Solutions', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Plot 2: Convergence
ax = axes[0, 1]
ax.plot(hc_hist, label='Hill Climbing', linewidth=2)
ax.plot(sa_hist['best'], label='Simulated Annealing', linewidth=2)
ax.set_xlabel('Iteration', fontweight='bold')
ax.set_ylabel('Best Value', fontweight='bold')
ax.set_title('Convergence Comparison', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Plot 3: SA Temperature
ax = axes[1, 0]
ax.plot(sa_hist['temperature'], color='red', linewidth=2)
ax.set_xlabel('Iteration', fontweight='bold')
ax.set_ylabel('Temperature', fontweight='bold')
ax.set_title('Temperature Schedule', fontweight='bold')
ax.set_yscale('log')
ax.grid(alpha=0.3)

# Plot 4: SA Current vs Best
ax = axes[1, 1]
ax.plot(sa_hist['current'], alpha=0.5, label='Current solution', linewidth=1)
ax.plot(sa_hist['best'], label='Best so far', linewidth=2, color='red')
ax.set_xlabel('Iteration', fontweight='bold')
ax.set_ylabel('Objective Value', fontweight='bold')
ax.set_title('SA: Exploration vs Exploitation', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("SA can escape local optima by accepting worse moves when T is high!")

## Part 4: Traveling Salesman Problem (TSP)

### Classic Optimization Problem

**Problem**: Find shortest route visiting all cities exactly once.

**Why hard?**: n! possible routes for n cities
- 10 cities: 3.6 million routes
- 20 cities: 2.4 × 10¹⁸ routes

**Applications**: Delivery routing, PCB drilling, DNA sequencing


In [ ]:
class TSP:
    """Traveling Salesman Problem."""
    
    def __init__(self, cities: np.ndarray):
        """
        Initialize TSP.
        
        Args:
            cities: Array of city coordinates (n_cities, 2)
        """
        self.cities = cities
        self.n_cities = len(cities)
        
        # Precompute distance matrix
        self.distances = np.zeros((self.n_cities, self.n_cities))
        for i in range(self.n_cities):
            for j in range(self.n_cities):
                self.distances[i, j] = np.linalg.norm(cities[i] - cities[j])
    
    def tour_length(self, tour: List[int]) -> float:
        """Calculate total tour length."""
        length = 0
        for i in range(len(tour)):
            length += self.distances[tour[i], tour[(i + 1) % len(tour)]]
        return length
    
    def random_tour(self) -> List[int]:
        """Generate random tour."""
        tour = list(range(self.n_cities))
        random.shuffle(tour)
        return tour
    
    def get_neighbors_2opt(self, tour: List[int]) -> List[List[int]]:
        """Generate neighbors using 2-opt: reverse a segment."""
        neighbors = []
        for i in range(len(tour) - 1):
            for j in range(i + 2, len(tour)):
                # Create neighbor by reversing segment [i+1:j]
                neighbor = tour[:i+1] + tour[i+1:j+1][::-1] + tour[j+1:]
                neighbors.append(neighbor)
        return neighbors
    
    def get_random_neighbor_2opt(self, tour: List[int]) -> List[int]:
        """Generate single random 2-opt neighbor."""
        new_tour = tour.copy()
        i = random.randint(0, len(tour) - 2)
        j = random.randint(i + 2, len(tour))
        new_tour[i+1:j+1] = reversed(new_tour[i+1:j+1])
        return new_tour
    
    def visualize_tour(self, tour: List[int], title: str = "TSP Tour"):
        """Visualize a tour."""
        plt.figure(figsize=(10, 10))
        
        # Plot cities
        plt.scatter(self.cities[:, 0], self.cities[:, 1], 
                   s=200, c='red', zorder=5, edgecolors='black', linewidths=2)
        
        # Plot tour
        for i in range(len(tour)):
            start = self.cities[tour[i]]
            end = self.cities[tour[(i + 1) % len(tour)]]
            plt.arrow(start[0], start[1], 
                     end[0] - start[0], end[1] - start[1],
                     head_width=1, head_length=1, fc='blue', ec='blue',
                     length_includes_head=True, alpha=0.6, linewidth=2)
        
        # Label cities
        for i, city in enumerate(self.cities):
            plt.text(city[0], city[1], str(i), fontsize=12, 
                    ha='center', va='center', fontweight='bold', color='white')
        
        tour_len = self.tour_length(tour)
        plt.title(f'{title}\nLength: {tour_len:.2f}', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('X', fontweight='bold')
        plt.ylabel('Y', fontweight='bold')
        plt.grid(alpha=0.3)
        plt.axis('equal')
        plt.tight_layout()
        plt.show()


# Create TSP instance
print("Traveling Salesman Problem")
print("=" * 60)

# Generate random cities
np.random.seed(42)
n_cities = 20
cities = np.random.rand(n_cities, 2) * 100

tsp = TSP(cities)
print(f"Number of cities: {n_cities}")
print(f"Number of possible tours: {np.math.factorial(n_cities-1)//2:,}")
print()

# Random tour
random_tour = tsp.random_tour()
random_length = tsp.tour_length(random_tour)
print(f"Random tour length: {random_length:.2f}")

tsp.visualize_tour(random_tour, "Random Tour")

### Solve TSP with Local Search


In [ ]:
print("Solving TSP with Local Search Algorithms")
print("=" * 60)

# Initial tour
initial_tour = tsp.random_tour()
initial_length = tsp.tour_length(initial_tour)
print(f"Initial tour length: {initial_length:.2f}")
print()

# 1. Hill Climbing
print("1. Hill Climbing (2-opt)...")
hc = HillClimbing(tsp.tour_length, tsp.get_neighbors_2opt, maximize=False)
hc_tour, hc_length, hc_hist = hc.optimize(initial_tour.copy())
print(f"   Final length: {hc_length:.2f}")
print(f"   Improvement: {(initial_length - hc_length) / initial_length * 100:.1f}%")
print()

# 2. Simulated Annealing
print("2. Simulated Annealing...")
sa = SimulatedAnnealing(
    tsp.tour_length,
    tsp.get_random_neighbor_2opt,
    initial_temp=1000.0,
    cooling_rate=0.995,
    maximize=False
)
sa_tour, sa_length, sa_hist = sa.optimize(initial_tour.copy(), max_iterations=10000)
print(f"   Final length: {sa_length:.2f}")
print(f"   Improvement: {(initial_length - sa_length) / initial_length * 100:.1f}%")
print()

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot tours
for ax, tour, length, title in [
    (axes[0], initial_tour, initial_length, 'Initial'),
    (axes[1], hc_tour, hc_length, 'Hill Climbing'),
    (axes[2], sa_tour, sa_length, 'Simulated Annealing')
]:
    # Plot cities
    ax.scatter(cities[:, 0], cities[:, 1], s=100, c='red', 
              zorder=5, edgecolors='black', linewidths=1)
    
    # Plot tour
    tour_coords = np.array([cities[i] for i in tour + [tour[0]]])
    ax.plot(tour_coords[:, 0], tour_coords[:, 1], 
           'b-', linewidth=2, alpha=0.6)
    
    ax.set_title(f'{title}\nLength: {length:.2f}', 
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X', fontweight='bold')
    ax.set_ylabel('Y', fontweight='bold')
    ax.grid(alpha=0.3)
    ax.axis('equal')

plt.tight_layout()
plt.show()

# Plot convergence
plt.figure(figsize=(12, 6))
plt.plot([initial_length] * len(hc_hist), 'k--', label='Initial', linewidth=2)
plt.plot(hc_hist, label='Hill Climbing', linewidth=2)
plt.plot(sa_hist['best'], label='Simulated Annealing', linewidth=2)
plt.xlabel('Iteration', fontweight='bold', fontsize=12)
plt.ylabel('Tour Length', fontweight='bold', fontsize=12)
plt.title('TSP Solution Convergence', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Best solution: Simulated Annealing with length {sa_length:.2f}")

## Part 5: N-Queens Problem

### Classic Constraint Satisfaction Problem

**Problem**: Place N queens on N×N chessboard so no queen attacks another.

**Constraints**:
- No two queens in same row
- No two queens in same column
- No two queens in same diagonal


In [ ]:
class NQueens:
    """N-Queens problem solver."""
    
    def __init__(self, n: int):
        """
        Initialize N-Queens.
        
        Args:
            n: Board size (n x n)
        """
        self.n = n
    
    def conflicts(self, state: List[int]) -> int:
        """
        Count number of conflicts (attacking pairs).
        State: list where state[i] = column of queen in row i
        """
        conflicts = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                # Same column?
                if state[i] == state[j]:
                    conflicts += 1
                # Same diagonal?
                if abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        return conflicts
    
    def random_state(self) -> List[int]:
        """Generate random state."""
        return [random.randint(0, self.n - 1) for _ in range(self.n)]
    
    def get_neighbors(self, state: List[int]) -> List[List[int]]:
        """Generate all neighbors (move one queen in its row)."""
        neighbors = []
        for row in range(self.n):
            for col in range(self.n):
                if col != state[row]:
                    neighbor = state.copy()
                    neighbor[row] = col
                    neighbors.append(neighbor)
        return neighbors
    
    def get_random_neighbor(self, state: List[int]) -> List[int]:
        """Generate random neighbor."""
        neighbor = state.copy()
        row = random.randint(0, self.n - 1)
        col = random.randint(0, self.n - 1)
        neighbor[row] = col
        return neighbor
    
    def visualize(self, state: List[int], title: str = "N-Queens"):
        """Visualize board state."""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Draw board
        for i in range(self.n):
            for j in range(self.n):
                color = 'wheat' if (i + j) % 2 == 0 else 'tan'
                ax.add_patch(plt.Rectangle((j, self.n - 1 - i), 1, 1, 
                                          facecolor=color, edgecolor='black'))
        
        # Draw queens
        for row, col in enumerate(state):
            ax.text(col + 0.5, self.n - 1 - row + 0.5, '♛', 
                   fontsize=40, ha='center', va='center')
        
        conflicts = self.conflicts(state)
        ax.set_xlim(0, self.n)
        ax.set_ylim(0, self.n)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title(f'{title} ({self.n}x{self.n})\nConflicts: {conflicts}',
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()


# Solve N-Queens
print("N-Queens Problem")
print("=" * 60)

n = 8
nqueens = NQueens(n)

# Random initial state
initial_state = nqueens.random_state()
initial_conflicts = nqueens.conflicts(initial_state)
print(f"Initial conflicts: {initial_conflicts}")

nqueens.visualize(initial_state, "Initial State")

In [ ]:
# Solve with local search
print("Solving 8-Queens with Local Search")
print("=" * 60)

# Hill Climbing with random restart
print("Random Restart Hill Climbing...")
rrhc = RandomRestartHillClimbing(
    nqueens.conflicts,
    nqueens.get_neighbors,
    nqueens.random_state,
    maximize=False
)

hc_state, hc_conflicts, hc_histories = rrhc.optimize(n_restarts=10, max_iterations=1000)
print(f"Final conflicts: {hc_conflicts}")
if hc_conflicts == 0:
    print("✓ Solution found!")
print()

# Simulated Annealing
print("Simulated Annealing...")
sa = SimulatedAnnealing(
    nqueens.conflicts,
    nqueens.get_random_neighbor,
    initial_temp=100.0,
    cooling_rate=0.99,
    maximize=False
)

sa_state, sa_conflicts, sa_hist = sa.optimize(initial_state.copy(), max_iterations=5000)
print(f"Final conflicts: {sa_conflicts}")
if sa_conflicts == 0:
    print("✓ Solution found!")
print()

# Visualize best solutions
if hc_conflicts == 0:
    nqueens.visualize(hc_state, "Hill Climbing Solution")

if sa_conflicts == 0:
    nqueens.visualize(sa_state, "Simulated Annealing Solution")

# Plot convergence
plt.figure(figsize=(12, 6))
plt.plot(sa_hist['best'], label='Simulated Annealing', linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', label='Goal (0 conflicts)')
plt.xlabel('Iteration', fontweight='bold', fontsize=12)
plt.ylabel('Number of Conflicts', fontweight='bold', fontsize=12)
plt.title('N-Queens Solution Progress', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Temperature Schedule
Implement and compare different cooling schedules for simulated annealing:
- Linear: T = T_0 - α*t
- Exponential: T = T_0 * α^t  
- Logarithmic: T = T_0 / log(t+2)

In [ ]:
# TODO: Implement and compare cooling schedules
# Your code here
pass

### Exercise 2: TSP Neighborhoods
Implement additional TSP neighborhood structures:
- 3-opt: Remove 3 edges and reconnect
- Swap: Exchange positions of two cities

In [ ]:
# TODO: Implement additional neighborhoods and compare
# Your code here
pass

### Exercise 3: Performance Analysis
Run each algorithm 30 times and analyze:
- Mean solution quality
- Standard deviation
- Success rate
- Average runtime

In [ ]:
# TODO: Statistical performance analysis
# Your code here
pass

## Summary

### Key Takeaways

1. **Local Search** - Navigate search space via neighbors
2. **Hill Climbing** - Simple but gets stuck in local optima
3. **Random Restart** - Multiple tries improve success
4. **Simulated Annealing** - Accept worse moves to escape local optima
5. **Problem Representation** - Neighbor function is crucial
6. **TSP and N-Queens** - Classic optimization problems

### Algorithm Comparison

| Algorithm | Pros | Cons | Best For |
|-----------|------|------|----------|
| Hill Climbing | Fast, simple | Stuck in local optima | Good initial solutions |
| Random Restart | Better coverage | More computation | Medium-sized problems |
| Simulated Annealing | Escapes local optima | Needs tuning | Complex landscapes |

### Next Steps

In Lab 3, we'll learn about **Constraint Satisfaction Problems** - a powerful framework for systematic optimization!
